In [1]:
import os 
os.chdir("/app/")
### data processing

from commonPyLib.helpers.file import AppFolderPath, FileHelper
from src.mlResolver.processDataToGraphData import processDataToGraphData
from src.mlResolver.createChart import createCharts
from src.mlResolver.chatTransformer import chartDataTransformers


projectName = "basic5D-5"
projectFolderPath, chartsFolderPath, modelsFolderPath, rawDataFolder, processedDataFile = AppFolderPath.getMLChartProjectFolderPath(projectName)


folderPaths = [projectFolderPath, chartsFolderPath, modelsFolderPath]
for folderPath in folderPaths:
    FileHelper.createFolderIfNotExists(folderPath)


lookbackDays = 5
futureDays = 5
isDebug = False


graphData = {}
for csvFile in [f for f in os.listdir(rawDataFolder) if f.endswith('.csv')]:
    rawDataPath = f"{rawDataFolder}/{csvFile}"
    tickerGraphData = processDataToGraphData(csvFile, rawDataPath, lookbackDays, futureDays, isDebug=isDebug)
    graphData = {**graphData, **tickerGraphData}

createCharts(graphData, lookbackDays, chartsFolderPath, isDebug=isDebug, removeExistingFiles=False, isReCreate=False)
processedMatrixData = chartDataTransformers(lookbackDays, graphData, chartsFolderPath)
FileHelper.saveToJson(processedDataFile, processedMatrixData, indent=None)


In [ ]:
## Training
import os 
os.chdir("/app/")

from src.mlResolver.trainner import modelTrainer
from src.mlResolver.models.cnn5to5 import Model
from commonPyLib.helpers.file import AppFolderPath, FileHelper


projectName = "basic5D-5"
modelName = "cnn20250218_5_5_mix.pth"

projectFolderPath, chartsFolderPath, modelsFolderPath, rawDataFolder, processedDataFile = AppFolderPath.getMLChartProjectFolderPath(projectName)
processedData = FileHelper.loadFromJson(processedDataFile)
modelPath = f"{modelsFolderPath}/{modelName}"
model = Model()

epochs = 40
learningRate = 0.0001
batchSize = 32
patience = 5

modelTrainer(model, processedData, epochs, learningRate, batchSize, patience, modelPath)


In [ ]:
import shutil
import os

## move model to models folder

sourceModelPath = modelPath
destModelPath = f"{AppFolderPath.mlChart.models}/{modelName}"

try:
    shutil.move(sourceModelPath, destModelPath)
    print(f"Successfully moved model from {sourceModelPath} to {destModelPath}")
except Exception as e:
    print(f"Error moving file: {str(e)}")


In [22]:
# dateback prediction
import requests
from commonPyLib.fmp import FMP


def guess(tickers: list[str], asOfDate: str, predictionDate: str):
    url = "http://0.0.0.0:5000/getPrediction"

    totalGuesses = 0
    totalCorrect = 0
    stockReturns = []

    data = []

    for ticker in tickers:
        print(f"Processing {ticker}...")
        response = requests.post(url, json={"symbol": ticker, "asOfDate": asOfDate})
        prices = FMP.getTickerHistoricalPrices(ticker, asOfDate, predictionDate)
        startPrice = prices[0].close
        endPrice = prices[-1].close
        change = endPrice - startPrice
        changeRate = change / startPrice
        changeRatePercent = f"{changeRate:.2%}"
        d = response.json()
        correctGuess = False
        upProbability = d['upProbability']
        downProbability = d['downProbability']
        totalGuesses += 1
        if upProbability > downProbability:
            if change >= 0: 
                totalCorrect += 1
                correctGuess = True
            stockReturns.append(changeRate)
        else:
            if change < 0: 
                totalCorrect += 1
                correctGuess = True
            stockReturns.append(-changeRate)

        obj = {
            "ticker": ticker,
            "Up vs Down": f"{upProbability:.2%} vs {downProbability:.2%}",
            "Start to End Price": f"{startPrice} -> {endPrice}",
            "Change": f"{round(change, 2)} ({changeRatePercent})",
            "Good Guess?": correctGuess
        }
        data.append(obj)
    print("--------------------------------")
    print(f"use date: {asOfDate} to {predictionDate}")
    print(f"Total Guesses: {totalGuesses}")
    print(f"Total Correct: {totalCorrect}")
    print(f"Accuracy: {totalCorrect / totalGuesses:.2%}")
    avgReturn = sum(stockReturns) / len(stockReturns)
    print(f"Average Return: {avgReturn:.2%}")
    return data 


asOfDate = "2025-02-21"
predictionDate = "2025-02-28"

tickersOriginal = [
    "AAPL",
    "TSLA",
    "BABA",
    "NVDA",
    "UPST",
    "MSTR",
    "ARM",
    "QQQ",
    "AMD",
    "PDD",
    "CRWD",
    "PLTR",
    "Z",
    "SYM",
    "SOFI",
    "U"
]
tickersNon = ["MSFT", "HP", "GTLB", "MS", "WMT", ]
df = guess(tickersOriginal + tickersNon, asOfDate, predictionDate)

import pandas as pd
df = pd.DataFrame(df)
df


